Load Data

pip3 install pandas / install pandas

pip3 install numpy / install numpy

In [1]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('online_retail.csv', encoding='ISO-8859-1')

print("Dataset Shape:", df.shape)
print("\n" + "="*50)

print("Dataset Info:")
print(df.info()) # Displaying the structure of the DataFrame
print("\n" + "="*50)

print("Statistical Summary:")
print(df.describe(include='all')) # Displaying summary statistics
print("\n" + "="*50)

# print("Missing Values:")
# print(df.isnull().sum())
# print("\n" + "="*50)

# print("Duplicate Rows:", df.duplicated().sum())

Dataset Shape: (1067371, 8)

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB
None

Statistical Summary:
        Invoice StockCode                         Description      Quantity  \
count   1067371   1067371                             1062989  1.067371e+06   
unique    53628      5305                                5698           NaN   
top      537434    85123A  WHITE HANGING HEART T-LIGHT HOLDER           NaN   